# Make the final SXDS catalogue

We want to take all the columns. We will save a catalogue for each patch which can be stacked to produce the final catalogues for serving publicly. The is_inner flags are used to remove the overlap regions. As far as possible we try to mimic the tables produced for HSC PDR2.

In [1]:
import os
import json
import numpy as np
import lsst.daf.persistence as dafPersist
import time
import gc

from astropy.table import Table, join, vstack
import astropy.units as u

import warnings
warnings.filterwarnings("ignore")

In [2]:
hscBands = ['G', 'R', 'I', 'Z', 'Y']
vistaBands = ['Z', 'Y', 'J', 'H', 'Ks']
allBands = ['HSC-' +b for b in hscBands] + ['VISTA-' +b for b in vistaBands]

In [3]:
allBands

['HSC-G',
 'HSC-R',
 'HSC-I',
 'HSC-Z',
 'HSC-Y',
 'VISTA-Z',
 'VISTA-Y',
 'VISTA-J',
 'VISTA-H',
 'VISTA-Ks']

In [4]:
hscBands

['G', 'R', 'I', 'Z', 'Y']

In [5]:
os.getcwd()

'/Users/rs548/GitHub/lsst-ir-fusion/dmu5/dmu5_SXDS'

In [6]:
if os.getcwd()=='/Users/rs548/GitHub/lsst-ir-fusion/dmu5/dmu5_SXDS':
    BUTLER_LOC = '/Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data'
    DATA = '/Volumes/Raph500/lsst-ir-fusion/dmu5/dmu5_Example/data'
else:
    BUTLER_LOC = '../../dmu4/dmu4_SXDS/data'
    DATA =  './data'
butler =  dafPersist.Butler(inputs='{}/rerun/coaddForcedPhot'.format(BUTLER_LOC))

In [7]:
BUTLER_LOC, DATA

('/Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data',
 '/Volumes/Raph500/lsst-ir-fusion/dmu5/dmu5_Example/data')

In [8]:
full_patch_dict = json.load(open('../../dmu4/dmu4_SXDS/full_patch_dict.json', 'r'))

In [9]:
json.dumps(full_patch_dict)

'{"8282": ["0,8"], "8283": ["0,8", "1,7", "1,8", "2,7", "2,8", "3,7", "3,8", "4,7", "4,8", "5,7", "5,8", "6,7", "6,8", "7,7", "7,8", "8,8"], "8284": ["8,8"], "8523": ["0,1", "0,2", "0,3", "0,4", "0,5", "0,6", "0,7", "0,8", "1,1", "1,2", "1,3", "1,4", "1,5", "1,6", "1,7", "1,8", "2,1", "2,2", "2,3", "2,4", "2,5", "2,6", "2,7", "2,8", "3,1", "3,2", "3,3", "3,4", "3,5", "3,6", "3,7", "3,8", "4,1", "4,2", "4,3", "4,4", "4,5", "4,6", "4,7", "4,8", "5,2", "5,3", "5,4", "5,5", "5,6", "5,7", "5,8", "6,3", "6,4", "6,5", "6,6", "6,7", "6,8", "7,3", "7,4", "7,5", "7,6", "7,7", "7,8"], "8524": ["0,0", "0,1", "0,2", "0,3", "0,4", "0,5", "0,6", "0,7", "0,8", "1,0", "1,1", "1,2", "1,3", "1,4", "1,5", "1,6", "1,7", "1,8", "2,0", "2,1", "2,2", "2,3", "2,4", "2,5", "2,6", "2,7", "2,8", "3,0", "3,1", "3,2", "3,3", "3,4", "3,5", "3,6", "3,7", "3,8", "4,0", "4,1", "4,2", "4,3", "4,4", "4,5", "4,6", "4,7", "4,8", "5,0", "5,1", "5,2", "5,3", "5,4", "5,5", "5,6", "5,7", "5,8", "6,0", "6,1", "6,2", "6,3", "6,4

In [10]:
def addFlux(cat, sources, photoCalib):
    """Add magnitudes and fluxes to an astropy catalogues with instrument fluxes"""
    for c in cat.colnames:
        if (c.endswith('_instFlux')):
            try:
                mags = photoCalib.instFluxToMagnitude(sources, c.replace('_instFlux',''))
                flux = photoCalib.instFluxToNanojansky(sources, c.replace('_instFlux',''))
                cat["{}_mag".format(c.replace('_instFlux',''))] = mags[:,0]
                cat["{}_mag".format(c.replace('_instFlux',''))].unit = u.mag
                cat["{}_mag".format(c.replace('_instFlux',''))].description = cat[c].description.replace(
                    'instFlux', 'mag')
                
                cat["{}_magErr".format(c.replace('_instFlux',''))] = mags[:,1]
                cat["{}_magErr".format(c.replace('_instFlux',''))].unit = u.mag
                cat["{}_magErr".format(c.replace('_instFlux',''))].description = cat[c].description.replace(
                    'instFlux', 'mag')
                
                cat["{}_flux".format(c.replace('_instFlux',''))] = flux[:,0]
                cat["{}_flux".format(c.replace('_instFlux',''))].unit = u.nJy
                cat["{}_flux".format(c.replace('_instFlux',''))].description = cat[c].description.replace(
                    'instFlux', 'flux')
                
                cat["{}_fluxErr".format(c.replace('_instFlux',''))] = flux[:,1]
                cat["{}_fluxErr".format(c.replace('_instFlux',''))].unit = u.nJy
                cat["{}_fluxErr".format(c.replace('_instFlux',''))].description = cat[c].description.replace(
                    'instFlux', 'flux')
            except:
                pass
    return cat

def makeCat(tract, patch, BUTLER_LOC,DATA=DATA,writeBandCats=True):
    """make the final catalogue on a given patch for later stacking"""
    cat =Table()
    tract = int(tract)
    for band in allBands:
        #We must keep columns under 68 characters by replacing long names
        mapping = { 
            'SecondDerivative':'SD', 
            'DoubleShapelet':'DS',
            'badCentroid':'BC',
            'badInitialCentroid':'BIC'
        }

        try:
            CoaddCalexp = butler.get('deepCoadd_calexp',  {'filter': band, 'tract': tract, 'patch': patch})
            CoaddPhotoCalib = CoaddCalexp.getPhotoCalib()
        
            measSources = butler.get('deepCoadd_meas', {'filter': band, 'tract': tract, 'patch': patch})
            measCat = measSources.asAstropy()
            measCat = addFlux(measCat, measSources, CoaddPhotoCalib)
            for c in measCat.colnames:    
                if c != 'id':
                    measCat[c].name = "{}_{}_{}".format(band,'m', c)
                
            forcedSources = butler.get('deepCoadd_forced_src', {'filter': band, 'tract': tract, 'patch': patch})
            forcedCat = forcedSources.asAstropy()
            forcedCat = addFlux(forcedCat, forcedSources, CoaddPhotoCalib)
            for c in forcedCat.colnames:    
                if c != 'id':
                    forcedCat[c].name = "{}_{}_{}".format(band,'f', c)
                    
            bandCat = join(measCat,forcedCat,join_type='outer')
            
            for c in bandCat.colnames:    
                if c != 'id':
                    newName = c
                    for k in mapping:
                        newName = newName.replace(k, mapping[k])
                    bandCat[c].name = newName
                    if len(newName)>68:
                        print('Name too long for fits writing.')
            if writeBandCats:
                bandCat.write(DATA+'/{}_{}_{}_fullCat.fits'.format(band,tract,patch), overwrite=True)
                
            if len(cat)==0:
                #On first band no join
                cat = bandCat
            else:
                #After first band join tables in
                cat = join(cat, bandCat,join_type='outer')
  
            
        except:
            warnings.warn("Band {} failed.".format(band))
                

    if len(cat) == 0:
        cat=None
    return cat.copy()

#makeCat(8524, '3,3', BUTLER_LOC)

In [13]:
cat = makeCat(8524, '3,3', BUTLER_LOC)
#cat[cat.colnames[0:999]].write(DATA+'/full_cat_{}_{}.fits'.format(8524, '3,3'), overwrite=True)
print('success', len(cat.colnames))

DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-G/8524/3,3/meas-HSC-G-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/HSC-G/8524/3,3/forcedSrc-HSC-G-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-R/8524/3,3/meas-HSC-R-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/HSC-R/8524/3,3/forcedSrc-HSC-R-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-I/8524/3,3/meas-HSC-I-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/HSC-I/8524/3,3/forcedSrc-HSC-I-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-Z/8524/3,3/meas-HSC-Z-8524-3,3.fits
DEBUG:  

In [14]:
cat.colnames[0:5]

['id',
 'HSC-G_m_coord_ra',
 'HSC-G_m_coord_dec',
 'HSC-G_m_parent',
 'HSC-G_m_merge_footprint_HSC-G']

In [15]:
cat['HSC-G_m_slot_CalibFlux_instFlux'].description.replace('instFlux', 'mag')

'mag within 12.000000-pixel aperture'

In [16]:
cat['HSC-G_m_slot_CalibFlux_mag'].description

'mag within 12.000000-pixel aperture'

In [17]:
cols = Table()
cols['name'] = cat.colnames
cols['description'] = [cat[c].description for c in cat.colnames]
cols['unit'] = [str(cat[c].unit) for c in cat.colnames]
cols['type'] = [cat[c].dtype for c in cat.colnames]
cols.write('./columns_descriptions.csv')

In [18]:
reduced_cols = [ 'id', 'VISTA-Ks_m_coord_ra', 'VISTA-Ks_m_coord_dec']
for aper in ['6', '9', '12', '17']:
    reduced_cols += ['{}_m_base_CircularApertureFlux_{}_0_mag'.format(b,aper) for b in allBands]
    reduced_cols += ['{}_m_base_CircularApertureFlux_{}_0_magErr'.format(b,aper) for b in allBands]
    reduced_cols += ['{}_m_base_CircularApertureFlux_{}_0_flag'.format(b,aper) for b in allBands]
    
reduced_cols += ['{}_m_base_PsfFlux_apCorr'.format(b) for b in allBands]
reduced_cols

['id',
 'VISTA-Ks_m_coord_ra',
 'VISTA-Ks_m_coord_dec',
 'HSC-G_m_base_CircularApertureFlux_6_0_mag',
 'HSC-R_m_base_CircularApertureFlux_6_0_mag',
 'HSC-I_m_base_CircularApertureFlux_6_0_mag',
 'HSC-Z_m_base_CircularApertureFlux_6_0_mag',
 'HSC-Y_m_base_CircularApertureFlux_6_0_mag',
 'VISTA-Z_m_base_CircularApertureFlux_6_0_mag',
 'VISTA-Y_m_base_CircularApertureFlux_6_0_mag',
 'VISTA-J_m_base_CircularApertureFlux_6_0_mag',
 'VISTA-H_m_base_CircularApertureFlux_6_0_mag',
 'VISTA-Ks_m_base_CircularApertureFlux_6_0_mag',
 'HSC-G_m_base_CircularApertureFlux_6_0_magErr',
 'HSC-R_m_base_CircularApertureFlux_6_0_magErr',
 'HSC-I_m_base_CircularApertureFlux_6_0_magErr',
 'HSC-Z_m_base_CircularApertureFlux_6_0_magErr',
 'HSC-Y_m_base_CircularApertureFlux_6_0_magErr',
 'VISTA-Z_m_base_CircularApertureFlux_6_0_magErr',
 'VISTA-Y_m_base_CircularApertureFlux_6_0_magErr',
 'VISTA-J_m_base_CircularApertureFlux_6_0_magErr',
 'VISTA-H_m_base_CircularApertureFlux_6_0_magErr',
 'VISTA-Ks_m_base_Circul

## Run over all patches
Now we simply loop over every patch to make the full field. This will only work on a small number of patches. The functions developed here will be added to obs_vista in due course for creating job arrays.

In [19]:
set(reduced_cols).intersection(set(cat.colnames))

{'HSC-G_m_base_CircularApertureFlux_12_0_flag',
 'HSC-G_m_base_CircularApertureFlux_12_0_mag',
 'HSC-G_m_base_CircularApertureFlux_12_0_magErr',
 'HSC-G_m_base_CircularApertureFlux_17_0_flag',
 'HSC-G_m_base_CircularApertureFlux_17_0_mag',
 'HSC-G_m_base_CircularApertureFlux_17_0_magErr',
 'HSC-G_m_base_CircularApertureFlux_6_0_flag',
 'HSC-G_m_base_CircularApertureFlux_6_0_mag',
 'HSC-G_m_base_CircularApertureFlux_6_0_magErr',
 'HSC-G_m_base_CircularApertureFlux_9_0_flag',
 'HSC-G_m_base_CircularApertureFlux_9_0_mag',
 'HSC-G_m_base_CircularApertureFlux_9_0_magErr',
 'HSC-G_m_base_PsfFlux_apCorr',
 'HSC-I_m_base_CircularApertureFlux_12_0_flag',
 'HSC-I_m_base_CircularApertureFlux_12_0_mag',
 'HSC-I_m_base_CircularApertureFlux_12_0_magErr',
 'HSC-I_m_base_CircularApertureFlux_17_0_flag',
 'HSC-I_m_base_CircularApertureFlux_17_0_mag',
 'HSC-I_m_base_CircularApertureFlux_17_0_magErr',
 'HSC-I_m_base_CircularApertureFlux_6_0_flag',
 'HSC-I_m_base_CircularApertureFlux_6_0_mag',
 'HSC-I_m_b

In [20]:
reduced_cat = Table()
mask = cat['VISTA-Ks_m_detect_isPatchInner'] & cat['VISTA-Ks_m_detect_isTractInner']
reduced_cat = vstack([reduced_cat, cat[mask][list(set(reduced_cols).intersection(set(cat.colnames)) )]])

In [21]:
reduced_cat[:5]

VISTA-J_m_base_CircularApertureFlux_17_0_flag,HSC-G_m_base_CircularApertureFlux_6_0_magErr,VISTA-J_m_base_CircularApertureFlux_9_0_magErr,VISTA-J_m_base_CircularApertureFlux_6_0_mag,HSC-G_m_base_CircularApertureFlux_9_0_mag,VISTA-J_m_base_CircularApertureFlux_6_0_magErr,HSC-Y_m_base_CircularApertureFlux_12_0_magErr,VISTA-Y_m_base_CircularApertureFlux_9_0_magErr,HSC-G_m_base_CircularApertureFlux_9_0_flag,HSC-Y_m_base_CircularApertureFlux_9_0_mag,VISTA-Y_m_base_PsfFlux_apCorr,HSC-Z_m_base_CircularApertureFlux_12_0_flag,VISTA-J_m_base_CircularApertureFlux_12_0_flag,VISTA-Ks_m_base_CircularApertureFlux_9_0_magErr,HSC-Z_m_base_CircularApertureFlux_9_0_flag,HSC-I_m_base_CircularApertureFlux_12_0_flag,VISTA-H_m_base_CircularApertureFlux_6_0_flag,HSC-G_m_base_CircularApertureFlux_17_0_magErr,HSC-R_m_base_CircularApertureFlux_6_0_flag,HSC-Y_m_base_PsfFlux_apCorr,VISTA-Ks_m_base_CircularApertureFlux_17_0_flag,VISTA-Ks_m_base_CircularApertureFlux_17_0_magErr,HSC-I_m_base_PsfFlux_apCorr,VISTA-J_m_base_CircularApertureFlux_12_0_magErr,HSC-R_m_base_CircularApertureFlux_6_0_magErr,HSC-Z_m_base_CircularApertureFlux_17_0_flag,HSC-Z_m_base_CircularApertureFlux_17_0_magErr,HSC-R_m_base_CircularApertureFlux_17_0_mag,HSC-Y_m_base_CircularApertureFlux_12_0_mag,VISTA-Y_m_base_CircularApertureFlux_6_0_magErr,HSC-I_m_base_CircularApertureFlux_12_0_mag,HSC-Z_m_base_CircularApertureFlux_9_0_mag,VISTA-Ks_m_base_CircularApertureFlux_12_0_flag,HSC-R_m_base_PsfFlux_apCorr,VISTA-H_m_base_CircularApertureFlux_17_0_flag,VISTA-Y_m_base_CircularApertureFlux_17_0_flag,HSC-G_m_base_CircularApertureFlux_6_0_mag,HSC-R_m_base_CircularApertureFlux_9_0_magErr,HSC-I_m_base_CircularApertureFlux_17_0_flag,VISTA-Ks_m_base_CircularApertureFlux_6_0_magErr,HSC-G_m_base_CircularApertureFlux_9_0_magErr,VISTA-Ks_m_base_CircularApertureFlux_9_0_flag,HSC-Z_m_base_CircularApertureFlux_12_0_magErr,HSC-I_m_base_CircularApertureFlux_17_0_mag,VISTA-Ks_m_base_CircularApertureFlux_9_0_mag,HSC-I_m_base_CircularApertureFlux_6_0_mag,VISTA-Ks_m_base_PsfFlux_apCorr,HSC-R_m_base_CircularApertureFlux_9_0_mag,HSC-Y_m_base_CircularApertureFlux_6_0_flag,VISTA-Ks_m_coord_ra,HSC-I_m_base_CircularApertureFlux_9_0_mag,HSC-G_m_base_PsfFlux_apCorr,HSC-Y_m_base_CircularApertureFlux_9_0_magErr,HSC-R_m_base_CircularApertureFlux_12_0_mag,HSC-R_m_base_CircularApertureFlux_12_0_magErr,VISTA-J_m_base_CircularApertureFlux_12_0_mag,HSC-R_m_base_CircularApertureFlux_6_0_mag,HSC-G_m_base_CircularApertureFlux_12_0_magErr,HSC-G_m_base_CircularApertureFlux_6_0_flag,HSC-R_m_base_CircularApertureFlux_9_0_flag,HSC-R_m_base_CircularApertureFlux_12_0_flag,HSC-R_m_base_CircularApertureFlux_17_0_magErr,VISTA-J_m_base_CircularApertureFlux_6_0_flag,VISTA-Y_m_base_CircularApertureFlux_9_0_mag,VISTA-J_m_base_CircularApertureFlux_9_0_flag,VISTA-Ks_m_base_CircularApertureFlux_12_0_magErr,VISTA-Ks_m_base_CircularApertureFlux_6_0_flag,HSC-Y_m_base_CircularApertureFlux_6_0_magErr,HSC-Z_m_base_CircularApertureFlux_6_0_magErr,HSC-I_m_base_CircularApertureFlux_17_0_magErr,VISTA-H_m_base_CircularApertureFlux_6_0_magErr,VISTA-H_m_base_CircularApertureFlux_9_0_mag,HSC-Z_m_base_CircularApertureFlux_12_0_mag,VISTA-Y_m_base_CircularApertureFlux_17_0_mag,HSC-Z_m_base_CircularApertureFlux_17_0_mag,HSC-I_m_base_CircularApertureFlux_12_0_magErr,HSC-G_m_base_CircularApertureFlux_12_0_mag,VISTA-Y_m_base_CircularApertureFlux_12_0_flag,HSC-Z_m_base_CircularApertureFlux_9_0_magErr,VISTA-Y_m_base_CircularApertureFlux_9_0_flag,HSC-Y_m_base_CircularApertureFlux_17_0_magErr,VISTA-H_m_base_CircularApertureFlux_9_0_magErr,HSC-I_m_base_CircularApertureFlux_6_0_magErr,VISTA-J_m_base_CircularApertureFlux_9_0_mag,HSC-Y_m_base_CircularApertureFlux_17_0_mag,VISTA-Y_m_base_CircularApertureFlux_12_0_magErr,VISTA-Y_m_base_CircularApertureFlux_6_0_flag,HSC-G_m_base_CircularApertureFlux_12_0_flag,VISTA-J_m_base_CircularApertureFlux_17_0_mag,HSC-Y_m_base_CircularApertureFlux_12_0_flag,VISTA-Ks_m_base_CircularApertureFlux_12_0_mag,HSC-Z_m_base_CircularApertureFlux_6_0_

In [22]:
name_base = DATA+'/full_cat_{}_{}_tab{}.fits'
reduced_cat = Table()
p_n=0
for tract in full_patch_dict:
    for patch in full_patch_dict[tract]:
        #print(type(tract), type(patch))
        #skip if file exists
        if os.path.exists(DATA+'/reduced_cat_{}_{}.fits'.format(tract,patch)):
            print('Tract {}, patch {} already done'.format(tract, patch))
            #continue
        try:
            cat = makeCat(tract, patch, BUTLER_LOC)
            #These commented lines will print the catalogue in 1000 column chunks keeping id
            #bounds = np.append(np.arange(0,len(cat.colnames),998),len(cat.colnames))
            #for n in np.arange(len(bounds)-1):
            #    #Write tables in 1000 columns blocks
            #    #print(bounds[n], bounds[n+1])
            #    sub_cols=list(set(cat.colnames[bounds[n]:bounds[n+1]]).intersection(set(['id'])) )
            #    cat[cat.colnames[bounds[n]:bounds[n+1]]].write(name_base.format(
            #        tract,patch,n
            #    ), overwrite=True)
              
                
            #mask = cat['VISTA-Ks_m_detect_isPatchInner'] & cat['VISTA-Ks_m_detect_isTractInner']
            #cat = cat[mask]
            #get the set of reduced column names that are available (maybe not all bands)
            intersect_red_cols =list(set(reduced_cols).intersection(set(cat.colnames)) )
        
            cat[sorted(intersect_red_cols, reverse=True)].write(
                DATA+'/reduced_cat_{}_{}.fits'.format(tract,patch), overwrite=True
            )
       
            p_n+=1
            #Try to fix memory issues:
            del cat
            gc.collect()
            time.sleep(1)
        except:
            print('Failed.')
            pass #
        

Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Failed.
Tract 8524, patch 3,3 already done
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-G/8524/3

DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/VISTA-J/8524/3,5/meas-VISTA-J-8524-3,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/VISTA-J/8524/3,5/forcedSrc-VISTA-J-8524-3,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/VISTA-H/8524/3,5/meas-VISTA-H-8524-3,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/VISTA-H/8524/3,5/forcedSrc-VISTA-H-8524-3,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/VISTA-Ks/8524/3,5/meas-VISTA-Ks-8524-3,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/VISTA-Ks/8524/3,5/forcedSrc-VISTA-Ks-8524-3,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-G/8524/3,6/meas-

DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-G/8524/4,5/meas-HSC-G-8524-4,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/HSC-G/8524/4,5/forcedSrc-HSC-G-8524-4,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-R/8524/4,5/meas-HSC-R-8524-4,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/HSC-R/8524/4,5/forcedSrc-HSC-R-8524-4,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-I/8524/4,5/meas-HSC-I-8524-4,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/HSC-I/8524/4,5/forcedSrc-HSC-I-8524-4,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-Z/8524/4,5/meas-HSC-Z-8524-4,5.fits
DEBUG:  

DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/HSC-Z/8524/5,5/forcedSrc-HSC-Z-8524-5,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/VISTA-Y/8524/5,5/meas-VISTA-Y-8524-5,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/VISTA-Y/8524/5,5/forcedSrc-VISTA-Y-8524-5,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/VISTA-J/8524/5,5/meas-VISTA-J-8524-5,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/VISTA-J/8524/5,5/forcedSrc-VISTA-J-8524-5,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/VISTA-H/8524/5,5/meas-VISTA-H-8524-5,5.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/VISTA-H/8524/5,5/force

In [23]:
p_n

9

In [30]:
#len(reduced_cat)

In [31]:
#reduced_cat[:5]

In [32]:
#reduced_cat.write(DATA+'/reduced_cat.fits', overwrite=True)

## Parellel code

Here we will make a single cell to be written to a script for parrellelising

In [50]:
job_id = 34

job_dict=json.loads(open('../../dmu4/dmu4_SXDS/slurm/patch_job_dict.json', 'r').read())

tract = job_dict[str(job_id)][0]
patch = job_dict[str(job_id)][1]

cat = makeCat(tract, patch, BUTLER_LOC)

intersect_red_cols =list(set(reduced_cols).intersection(set(cat.colnames)) )
        
cat[sorted(intersect_red_cols, reverse=True)].write(
    DATA+'/reduced_cat_{}_{}.fits'.format(tract,patch), overwrite=True
)



DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-G/8524/3,3/meas-HSC-G-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/HSC-G/8524/3,3/forcedSrc-HSC-G-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-R/8524/3,3/meas-HSC-R-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/HSC-R/8524/3,3/forcedSrc-HSC-R-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-I/8524/3,3/meas-HSC-I-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddForcedPhot/deepCoadd-results/HSC-I/8524/3,3/forcedSrc-HSC-I-8524-3,3.fits
DEBUG:  /Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data/rerun/coaddPhot/deepCoadd-results/HSC-Z/8524/3,3/meas-HSC-Z-8524-3,3.fits
DEBUG:  

In [46]:
job_id = 34

job_dict=json.loads(open('../../dmu4/dmu4_SXDS/slurm/patch_job_dict.json', 'r').read())

In [48]:
job_dict[str(job_id)]

[8524, '3,3']

In [45]:
for i in job_dict:
    if job_dict[i][0] ==8524 and job_dict[i][1] =='3,3':
        print(i)

34
